In [ ]:
import pandas as pd
import requests
import RiotAnalyzer as Ra
from datetime import datetime

In [ ]:
api_key_dhil = 'RGAPI-9c72c116-9d41-4245-8af9-e431b87bf6cc'
game_version = "13.17.1"

In [ ]:
header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9,id;q=0.8,ja;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": f"{api_key_dhil}"
}
url_template = "https://{region_code}.api.riotgames.com/lol/{endpoint}?{query}"

In [142]:
def get_puuid(name:str, region_code:str="NA1"):
    url = url_template.format(region_code=region_code, endpoint="summoner/v4/summoners/by-name/"+name, query="page=1")
    response = requests.get(url, headers=header)
    if response.status_code != 200:
        print("Error: ", response.status_code)
    else:
        name_data = response.json()
    puuid = name_data["puuid"]
    return puuid

    

In [143]:
def get_mastery(puuid:str, region_code:str="NA1"):
    url = url_template.format(region_code=region_code, endpoint=f"champion-mastery/v4/champion-masteries/by-puuid/{puuid}", query="page=1")
    response = requests.get(url, headers=header)
    if response.status_code != 200:
        print("Error: ", response.status_code)
    else:
        return response.json()

In [ ]:
ra = Ra.RiotAnalyzer(api_key_dhil, version=game_version)

In [ ]:
champ_dict = ra.get_champion_dict()

In [144]:
def search_mastery_by_summoner_name(name, region_code:str="NA1"):
    puuid = get_puuid(name, region_code)
    mastery = get_mastery(puuid,region_code)
    df = pd.DataFrame(mastery)
    df["championName"] = df["championId"].apply(lambda x: champ_dict[str(x)])
    df = df[["championName", "championLevel", "championPoints", "lastPlayTime"]]
    df["lastPlayDate"] = df["lastPlayTime"].apply(lambda x: datetime.fromtimestamp(x/1000))
    df["lastPlayTime"] = df["lastPlayTime"].apply(lambda x: datetime.fromtimestamp(x/1000).strftime("%b %d %H:%M"))
    df["percent"] = df["championPoints"]/df["championPoints"].sum()
    df["percent"] = df["percent"].apply(lambda x: "{:.2%}".format(x))
    df.index += 1
    df.index.name = "Rank"
    return df

In [174]:
df = search_mastery_by_summoner_name("Michaelhoops", "NA1")
df.head(15)

,championName,championLevel,championPoints,lastPlayTime,lastPlayDate,percent
Rank,,,,,,
1,Shaco,7,250653,Aug 26 13:36,2023-08-26 13:36:47,14.60%
2,Teemo,6,86199,Jul 09 16:04,2023-07-09 16:04:07,5.02%
3,Master Yi,7,81984,May 07 11:55,2022-05-07 11:55:49,4.78%
4,Alistar,7,68186,Sep 10 11:04,2022-09-10 11:04:45,3.97%
5,Shyvana,7,56884,Aug 16 11:07,2022-08-16 11:07:44,3.31%
6,Pantheon,7,54198,Aug 26 14:12,2023-08-26 14:12:23,3.16%
7,Fizz,5,52827,Mar 05 00:43,2022-03-05 00:43:50,3.08%
8,Shen,6,51628,Aug 06 01:02,2023-08-06 01:02:06,3.01%
9,Lux,7,50097,Jul 14 16:43,2023-07-14 16:43:00,2.92%


In [126]:
df[df["championLevel"] == 7]

,championName,championLevel,championPoints,lastPlayTime,percent
Rank,,,,,
1,Master Yi,7,506471,Jun 04 09:06,20.56%
2,Lux,7,350410,Jun 04 04:59,14.23%
3,Nunu & Willump,7,261787,Jun 04 09:39,10.63%


In [165]:
def print_full(x):
    pd.set_option('display.max_rows', 10000)
    display(x)
    pd.reset_option('display.max_rows')

In [172]:
print_full(df)

,championName,championLevel,championPoints,lastPlayTime,lastPlayDate,percent
Rank,,,,,,
1,Neeko,3,10350,Sep 01 00:21,2023-09-01 00:21:06,16.89%
2,Varus,3,8377,Sep 02 12:47,2023-09-02 12:47:16,13.67%
3,Poppy,2,4090,Aug 28 23:40,2023-08-28 23:40:58,6.67%
4,Rakan,2,3579,Aug 19 22:39,2023-08-19 22:39:50,5.84%
5,Ashe,2,3388,Aug 31 23:15,2023-08-31 23:15:52,5.53%
6,Milio,2,2534,Aug 02 11:33,2023-08-02 11:33:21,4.13%
7,Ezreal,2,1840,Aug 30 22:39,2023-08-30 22:39:33,3.00%
8,Thresh,2,1824,Aug 19 23:35,2023-08-19 23:35:55,2.98%
9,Jhin,2,1808,Aug 28 23:18,2023-08-28 23:18:58,2.95%


In [148]:
df


,championName,championLevel,championPoints,lastPlayTime,lastPlayDate,percent
Rank,,,,,,
7,Jhin,5,79690,Sep 06 13:15,2023-09-06 13:15:29,3.24%
5,Ezreal,5,202461,Sep 06 09:27,2023-09-06 09:27:28,8.22%
27,Sivir,3,9687,Aug 29 11:29,2023-08-29 11:29:58,0.39%
4,Caitlyn,5,244189,Aug 29 09:53,2023-08-29 09:53:48,9.91%
69,Nautilus,1,1418,Aug 25 04:14,2023-08-25 04:14:19,0.06%
...,...,...,...,...,...,...
104,Gnar,1,130,Feb 25 04:50,2019-02-25 04:50:32,0.01%
85,Kayn,1,631,Feb 02 05:20,2019-02-02 05:20:31,0.03%
103,Kog'Maw,1,134,Jan 29 06:36,2019-01-29 06:36:15,0.01%


In [85]:
def get_champion_dict(version="13.17.1"):
    """Gets a dictionary of champion names and their IDs

    Args:
        version (str): The version of the game to get the champion data for

    Returns:
        dict: A dictionary of champion names and their IDs
    """
    url = f"http://ddragon.leagueoflegends.com/cdn/{version}/data/en_US/champion.json"
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Error {response.status_code} when querying champion data")
    data = response.json()
    # Take data and put inside a pandas dataframe
    df = pd.DataFrame(data["data"]).T
    return df

In [139]:
cd = get_champion_dict()
cd.get('key')['Aatrox']

'266'

In [87]:
cd.loc["Neeko"]

version                                              13.17.1
id                                                     Neeko
key                                                      518
name                                                   Neeko
title                                  the Curious Chameleon
blurb      Hailing from a long lost tribe of vastaya, Nee...
info       {'attack': 1, 'defense': 1, 'magic': 9, 'diffi...
image      {'full': 'Neeko.png', 'sprite': 'champion2.png...
tags                                         [Mage, Support]
partype                                                 Mana
stats      {'hp': 610, 'hpperlevel': 104, 'mp': 450, 'mpp...
Name: Neeko, dtype: object

In [113]:
cd.loc["Vex"]['tags']

['Mage']

In [179]:
cd.to_csv('biden.csv', index=False, na_rep = 'biden')

In [185]:
cr = {}
cr['Aatrox'] = [1]
cr['Ahri'] = [3]
cr

{'Aatrox': [1], 'Ahri': [3]}